A gente tem dados. E agora?

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/cassiasamp/calculadora-de-imoveis-jun-20/master/coleta_de_dados/dados_mercado_livre.csv')

In [3]:
df

,zona,quartos,area,preco
0,sul,1.0,45.0,950
1,sul,3.0,217.0,7000
2,sul,4.0,320.0,7000
3,sul,5.0,398.0,7000
4,sul,NaN,185.0,7000
...,...,...,...,...
1003,oeste,2.0,71.0,1300
1004,oeste,2.0,200.0,3000
1005,oeste,1.0,40.0,1200
1006,oeste,2.0,62.0,1300


Jeito mais simples de advinhar o preço (por exemplo da zona sul com 1 quarto e 40 m²):
- regressão linear
- colocar a média
- média pela área 
- metro quadrado da zona sul e multiplicar depois

In [4]:
df['preco_m2'] = df['preco']/df['area']

In [5]:
df.head()

,zona,quartos,area,preco,preco_m2
0,sul,1.0,45.0,950,21.111111
1,sul,3.0,217.0,7000,32.258065
2,sul,4.0,320.0,7000,21.875000
3,sul,5.0,398.0,7000,17.587940
4,sul,NaN,185.0,7000,37.837838


In [6]:
# area de sul, 46m², 1.0 - qual que é o preço?
df_sul = df.query('zona == "sul"')

In [7]:
df_sul['zona'].unique()

array(['sul'], dtype=object)

In [8]:
# df.query?
df_sul = df.query('zona == "sul" and quartos == 1')
# outro jeito de fazer o filtro
# df[(df['zona'] == 'sul') & (df['quartos'] == 1)]

In [9]:
df_sul.describe().T

,count,mean,std,min,25%,50%,75%,max
quartos,22.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
area,22.0,68.772727,78.793239,15.000000,30.000000,40.000000,60.000000,350.000000
preco,22.0,2575.000000,3964.297512,400.000000,662.500000,1100.000000,2100.000000,15000.000000
preco_m2,22.0,32.306477,17.752248,13.333333,21.354167,24.272727,38.035714,83.333333


In [10]:
# comandos mágicos para comparar a execução
# %timeit?
# %time?

In [11]:
media_de_preco_m2 = df_sul['preco_m2'].mean()

In [12]:
# o preço que a gente advinhou
media_de_preco_m2*50
# o preço da olx = 2.600 (tem 1000 de condominio, pode ser que tenha sido um bom chute)

1615.3238340491303

In [13]:
media_de_preco_m2*40
# o preço da olx = 4.000 (não sabemos se tem condomíno incluso)

1292.2590672393042

Modelando o problema com regressão linear

In [14]:
from sklearn.linear_model import LinearRegression

- variaveis independentes - o  que a gente vai usar para prever
- variavel dependente - o que a gente quer prever

In [15]:
df.head()

,zona,quartos,area,preco,preco_m2
0,sul,1.0,45.0,950,21.111111
1,sul,3.0,217.0,7000,32.258065
2,sul,4.0,320.0,7000,21.875000
3,sul,5.0,398.0,7000,17.587940
4,sul,NaN,185.0,7000,37.837838


In [16]:
y = df['preco']

In [17]:
df.drop('preco', axis=1)
# columns='preco'
# X

,zona,quartos,area,preco_m2
0,sul,1.0,45.0,21.111111
1,sul,3.0,217.0,32.258065
2,sul,4.0,320.0,21.875000
3,sul,5.0,398.0,17.587940
4,sul,NaN,185.0,37.837838
...,...,...,...,...
1003,oeste,2.0,71.0,18.309859
1004,oeste,2.0,200.0,15.000000
1005,oeste,1.0,40.0,30.000000
1006,oeste,2.0,62.0,20.967742


In [18]:
# sul : 0, norte : 1, leste : 2, oeste : 3 #label enconding
df_onehot = pd.get_dummies(df) # one hot enconding

In [19]:
df_onehot = df_onehot.drop('preco', axis=1)

In [20]:
X = df_onehot

In [21]:
X.head()

,quartos,area,preco_m2,zona_leste,zona_norte,zona_oeste,zona_sul
0,1.0,45.0,21.111111,0,0,0,1
1,3.0,217.0,32.258065,0,0,0,1
2,4.0,320.0,21.875000,0,0,0,1
3,5.0,398.0,17.587940,0,0,0,1
4,NaN,185.0,37.837838,0,0,0,1


In [22]:
X = X.drop('preco_m2', axis=1)

In [23]:
reg = LinearRegression()

In [24]:
y.head()

0     950
1    7000
2    7000
3    7000
4    7000
Name: preco, dtype: int64

In [25]:
X = X.fillna(X.median())

In [26]:
X

,quartos,area,zona_leste,zona_norte,zona_oeste,zona_sul
0,1.0,45.0,0,0,0,1
1,3.0,217.0,0,0,0,1
2,4.0,320.0,0,0,0,1
3,5.0,398.0,0,0,0,1
4,2.0,185.0,0,0,0,1
...,...,...,...,...,...,...
1003,2.0,71.0,0,0,1,0
1004,2.0,200.0,0,0,1,0
1005,1.0,40.0,0,0,1,0
1006,2.0,62.0,0,0,1,0


In [27]:
# regressão linear é sensível a valores faltantes
reg.fit(X, y) # ajustamos o modelo aos dados

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [28]:
X.head()

,quartos,area,zona_leste,zona_norte,zona_oeste,zona_sul
0,1.0,45.0,0,0,0,1
1,3.0,217.0,0,0,0,1
2,4.0,320.0,0,0,0,1
3,5.0,398.0,0,0,0,1
4,2.0,185.0,0,0,0,1


In [29]:
# zona sul, area é 50, quartos 1 -- 1615
# zona sul, area é 40, quartos 1 -- 1292

# regressão linear é sensível a valores extremos ou outliers
reg.predict([[1, 50, 0, 0, 0, 1], [1, 40, 0, 0, 0, 1]])

array([13370.85625759, 13127.64634378])

In [30]:
# quartos, area, zona leste, norte, oeste e sul
reg.coef_

array([-7.49493437e+00,  2.43209914e+01, -3.56030043e+03, -4.73959312e+03,
        1.37551818e+01,  8.28613837e+03])

In [31]:
# onde cortamos o eixo y, constante b da equação linear
reg.intercept_

3876.163253467187

In [32]:
import numpy as np
type(np.nan)

float

In [33]:
# Exercício: fazer o train test split do sklearn para dividir os dados entre treino e teste

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, 
                                                        test_size=0.25, 
                                                        random_state=42)

In [49]:
X_treino

,quartos,area,zona_leste,zona_norte,zona_oeste,zona_sul
82,4.0,333.0,0,0,0,1
254,2.0,45.0,0,1,0,0
924,4.0,470.0,0,0,1,0
408,2.0,90.0,0,1,0,0
653,1.0,44.0,1,0,0,0
...,...,...,...,...,...,...
106,2.0,750.0,0,0,0,1
270,2.0,90.0,0,1,0,0
860,4.0,610.0,0,0,1,0
435,2.0,40.0,1,0,0,0


In [37]:
len(X)

1008

In [38]:
len(X_treino)/1008

0.75

In [39]:
len(X_teste)/1008

0.25

In [40]:
# testar outros modelos: k vizinhos mais próximos(knn), árvore de decisão, validação cruzada, score

In [51]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_treino, y_treino)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [52]:
X_teste.head()

,quartos,area,zona_leste,zona_norte,zona_oeste,zona_sul
938,3.0,300.0,0,0,1,0
630,2.0,60.0,1,0,0,0
682,3.0,142.0,1,0,0,0
514,1.0,40.0,1,0,0,0
365,2.0,40.0,0,1,0,0


In [54]:
y_teste[:5]

938    12000
630     1300
682     2300
514     1600
365      700
Name: preco, dtype: int64

In [58]:
# árvore de decisao
dt.score(X_teste, y_teste)

-1.7357059697243458

In [60]:
# regressao
reg.score(X_teste, y_teste) # r quadrado por debaixo dos panos

-1.7354065611845324

In [65]:
# vizinhos mais próximos - knn
from sklearn.neighbors import KNeighborsRegressor

knr = KNeighborsRegressor()
knr.fit(X_treino, y_treino)
y_pred = knr.predict(X_teste)
knr.score(X_teste, y_teste)

-0.2623418888971245

In [66]:
from sklearn.metrics import r2_score

r2_score(y_teste, y_pred)

-0.2623418888971245

In [67]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(knr, X_treino, y_treino, cv=5)

In [71]:
scores.mean()

-6.713650614835865